In [239]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [240]:
# Read the CSV files into DataFrames.
athletic_sales_2020_df = pd.read_csv('Resources/athletic_sales_2020.csv')
athletic_sales_2021_df = pd.read_csv('Resources/athletic_sales_2021.csv')

In [241]:
# Display the 2020 sales DataFrame
athletic_sales_2020_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [242]:
# Display the 2021 sales DataFrame
athletic_sales_2021_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,65,750,487500,121875.00,Outlet
1,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,51,233,11883,3208.41,Outlet
2,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,50,275,137500,82500.00,Outlet
3,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,47,77,3619,2714.25,Online
4,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,64,225,14400,5184.00,Online


#### Check the data types of each DataFrame

In [243]:
# Check the 2020 sales data types.
athletic_sales_2020_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [244]:
# Check the 2021 sales data types.
athletic_sales_2021_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

#### Combine the sales data by rows.

In [245]:
# Combine the 2020 and 2021 sales DataFrames on the rows using an inner join, and reset the index.col
combined_sales_df = pd.concat([athletic_sales_2020_df, athletic_sales_2021_df], axis='rows', join='inner').reset_index(drop=True)

In [246]:
# checking the 2020 shape: 

athletic_sales_2020_df.shape


(1297, 12)

In [247]:
# checking the 2021 shape:
athletic_sales_2021_df.shape

(8346, 12)

In [248]:
# confirming the combined 2020 + 2021 dataframes shape: 
combined_sales_df.shape

(9643, 12)

In [249]:
# Check if any values are null.
combined_sales_df.isnull().sum()

retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64

In [250]:
# Check the data type of each column
combined_sales_df.dtypes

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [251]:
# Convert the "invoice_date" to a datetime datatype
combined_sales_df['invoice_date'] = pd.to_datetime(combined_sales_df['invoice_date'])

/var/folders/5g/q4p5gs2n4236kzpt7r4nbsbh0000gn/T/ipykernel_57319/2172351529.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_sales_df['invoice_date'] = pd.to_datetime(combined_sales_df['invoice_date'])


In [252]:
# Confirm that the "invoice_date" data type has been changed.
# check the data type of "invoice_date" column
combined_sales_df.dtypes


retailer                    object
retailer_id                  int64
invoice_date        datetime64[ns]
region                      object
state                       object
city                        object
product                     object
price_per_unit               int64
units_sold                   int64
total_sales                  int64
operating_profit           float64
sales_method                object
dtype: object

### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [253]:
# Show the number products sold for region, state, and city.
regrouped_units_sold_df = combined_sales_df.groupby(['region', 'state', 'city'])[["units_sold"]].sum()

# Rename the sum to "Total_Products_Sold".
regrouped_units_sold_df = regrouped_units_sold_df.rename(columns={'units_sold': 'Total_Products_Sold'})

# Sort the results in ascending order to show the top five regions, including the state and city that have the greatest number of products sold.
regrouped_units_sold_df = regrouped_units_sold_df.sort_values(by=['Total_Products_Sold'], ascending=False)

# Show the top 5 results.
regrouped_units_sold_df.head()                        

Total_Products_Sold
region    state      city                              
Northeast New York   New York                    111954
South     Texas      Houston                      90322
West      California San Francisco                85478
                     Los Angeles                  76384
Southeast Florida    Miami                        73135

#### Using `pivot_table`

In [254]:

# Show the number products sold for region, state, and city.
pt_units_sold = combined_sales_df.pivot_table(index=['region', 'state', 'city'],
                                            values=['units_sold'],
                                            aggfunc='sum')

# Rename the "units_sold" column to "Total_Products_Sold"
pt_units_sold.rename(columns={'units_sold': 'Total_Products_Sold'}, inplace=True)

# Sort the results in ascending order to show the top five regions, including the state and city that have the greatest number of products sold 
pt_units_sold = pt_units_sold.sort_values(by=['Total_Products_Sold'], ascending=False)
# Show the top 5 results.
pt_units_sold.head()

Total_Products_Sold
region    state      city                              
Northeast New York   New York                    111954
South     Texas      Houston                      90322
West      California San Francisco                85478
                     Los Angeles                  76384
Southeast Florida    Miami                        73135

### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [255]:
# Show the total sales for the products sold for each region, state, and city.
regrouped_total_sales_df = combined_sales_df.groupby(['region', 'state', 'city'])[["total_sales"]].sum()


# Rename the "total_sales" column to "Total Sales"
regrouped_total_sales_df = regrouped_total_sales_df.rename(columns={'total_sales': 'Total_Sales'})

# Sort the results in ascending order to show the top five regions, including the state and city that generated the most sales.
regrouped_total_sales_df = regrouped_total_sales_df.sort_values(by=['Total_Sales'], ascending=False)
# Show the top 5 results.
regrouped_total_sales_df.head()

Total_Sales
region    state          city                      
Northeast New York       New York          39801235
West      California     San Francisco     33973228
Southeast Florida        Miami             31600863
          South Carolina Charleston        29285637
          Florida        Orlando           27682851

#### Using `pivot_table`

In [256]:
# Show the total sales for the products sold for each region, state, and city.
pt_total_sales = combined_sales_df.pivot_table(index=['region', 'state', 'city'],
                                               values=['total_sales'],
                                               aggfunc='sum')

# Optional: Rename the "total_sales" column to "Total Sales"
pt_total_sales = pt_total_sales.rename(columns={'total_sales': 'Total_Sales'})

# Sort the results in ascending order to show the top five regions, including the state and city that generated the most sales.
pt_total_sales = pt_total_sales.sort_values(by=['Total_Sales'], ascending=False)


# Show the top 5 results.
pt_total_sales.head()

Total_Sales
region    state          city                      
Northeast New York       New York          39801235
West      California     San Francisco     33973228
Southeast Florida        Miami             31600863
          South Carolina Charleston        29285637
          Florida        Orlando           27682851

### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [257]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_sales_df = combined_sales_df.groupby(['retailer', 'region', 'state', 'city'])[["total_sales"]].sum()

# Rename the "total_sales" column to "Total Sales"
retailer_sales_df = retailer_sales_df.rename(columns={'total_sales': 'Total_Sales'})

# Sort the results in ascending order to show the top five retailers along with their region, state, and city that generated the most sales.
retailer_sales_df = retailer_sales_df.sort_values(by=['Total_Sales'], ascending=False)

# Show the top 5 results.
retailer_sales_df.head()

,,,,Total_Sales
retailer,region,state,city,
West Gear,West,California,San Francisco,32794405
Kohl's,West,California,Los Angeles,25127160
Foot Locker,Northeast,New York,New York,25008568
West Gear,West,Washington,Seattle,24862675
Foot Locker,Southeast,South Carolina,Charleston,24822280


#### Using `pivot_table`

In [258]:
# Show the total sales for the products sold for each retailer, region, state, and city.
pt_retailer_sales = combined_sales_df.pivot_table(index=['retailer', 'region', 'state', 'city'],
                                                  values=['total_sales'],
                                                  aggfunc='sum')

# Optional: Rename the "total_sales" column to "Total Sales"
pt_retailer_sales = pt_retailer_sales.rename(columns={'total_sales': 'Total_Sales'})

# Sort the results in ascending order to show the top five retailers along with their region, state, and city that generated the most sales.
pt_retailer_sales = pt_retailer_sales.sort_values(by=['Total_Sales'], ascending=False)
# Show the top 5 results.
pt_retailer_sales.head()

,,,,Total_Sales
retailer,region,state,city,
West Gear,West,California,San Francisco,32794405
Kohl's,West,California,Los Angeles,25127160
Foot Locker,Northeast,New York,New York,25008568
West Gear,West,Washington,Seattle,24862675
Foot Locker,Southeast,South Carolina,Charleston,24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [259]:
# Filter the sales data to get the women's athletic footwear sales data.
w_footwear_sales_df = combined_sales_df.loc[combined_sales_df['product'] == 'Women\'s Athletic Footwear']
w_footwear_sales_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
21,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,36,281,10116,3742.92,Outlet
22,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,41,247,10127,4658.42,Online
23,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875.00,In-store
26,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,45,300,135000,47250.00,Outlet
27,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,34,90,3060,1254.60,Outlet


#### Using `groupby`

In [260]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
units_sales_df = w_footwear_sales_df.groupby(['retailer', 'region', 'state', 'city'])[["units_sold"]].sum()

# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
units_sales_df = units_sales_df.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'})

# Sort the results in ascending order to show the top five retailers along with their region, state, and city that sold the most women's athletic footwear.
units_sales_df = units_sales_df.sort_values(by = (['Womens_Footwear_Units_Sold']), ascending=False)

# Show the top 5 results.
units_sales_df.head()

,,,,Womens_Footwear_Units_Sold
retailer,region,state,city,
West Gear,West,California,San Francisco,12107
Foot Locker,Northeast,New York,New York,10996
Kohl's,West,California,Los Angeles,10826
Foot Locker,Southeast,South Carolina,Charleston,8814
Sports Direct,South,Texas,Dallas,8790


#### Using `pivot_table`

In [261]:
#CORRECT THIS ONE.

# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
pt_units__sales = w_footwear_sales_df.pivot_table(index=['retailer', 'region', 'state', 'city'],
                                                    values=['units_sold'],
                                                    aggfunc='sum')

# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
pt_units__sales.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'}, inplace=True)

# Inline comments in starter code file did not include the following line 
# but I am adding it so it matches the .groupby method example for this problem:

# Sort the results in ascending order to show the top five retailers along with their region, state, and city that sold the most women's athletic footwear.
pt_units__sales = pt_units__sales.sort_values(by = (['Womens_Footwear_Units_Sold']), ascending=False)

# Show the top 5 results.
pt_units__sales.head()

,,,,Womens_Footwear_Units_Sold
retailer,region,state,city,
West Gear,West,California,San Francisco,12107
Foot Locker,Northeast,New York,New York,10996
Kohl's,West,California,Los Angeles,10826
Foot Locker,Southeast,South Carolina,Charleston,8814
Sports Direct,South,Texas,Dallas,8790


### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [262]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
pt_day_sales = w_footwear_sales_df.pivot_table(index=['invoice_date'], 
                                                   values=['total_sales'], 
                                                   aggfunc='sum')

# Optional: Rename the "total_sales" column to "Total Sales"
pt_day_sales.rename(columns={'total_sales': 'Total Sales'}, inplace=True)

# Show the table.
pt_day_sales.head()

,Total Sales
invoice_date,
2020-01-04,402743
2020-01-05,141801
2020-01-11,129556
2020-01-17,173013
2020-01-22,388250


In [263]:
# Resample the pivot table into daily bins, and get the total sales for each day.
pt_day_sales = pt_day_sales.resample('D').sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
pt_day_sales = pt_day_sales.sort_values(by=['Total Sales'], ascending=False)
pt_day_sales.head(10)

,Total Sales
invoice_date,
2021-07-16,1521825
2021-12-16,1473497
2021-06-17,1376988
2021-08-17,1086294
2021-07-23,1021806
2021-11-17,1021145
2021-12-09,915011
2021-06-24,884238
2021-07-09,869054


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [264]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
pt_week_sales = pt_day_sales.resample('W').sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
pt_week_sales = pt_week_sales.sort_values(by=['Total Sales'], ascending=False)
pt_week_sales.head(10)

,Total Sales
invoice_date,
2021-12-19,3098970
2021-12-12,2922161
2021-07-11,2835078
2021-07-18,2801449
2021-11-14,2531721
2021-08-22,2491259
2021-08-15,2463941
2021-11-21,2449537
2021-05-16,2422132
